In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
import time
from pymongo import MongoClient


In [24]:
# df_integrated = pd.read_csv("../integrated_cleaned/integration_cleaned_definitive.csv")
# import os
# from dotenv import load_dotenv

# load_dotenv(".env")

# connection_string = os.getenv("MONGODB_CONNECTION_STRING")
# client = MongoClient(connection_string)

# db = client.restaurant_db
# collection = db.restaurant_integrated
# documents = collection.find()
# data = [doc for doc in documents]
# df_integrated = pd.DataFrame(data)
df_integrated = pd.read_csv("../integrated_cleaned/integration_cleaned_definitive.csv")
df_integrated



,place_id,name,cook_type,types,address,latitude,longitude,rating,price_level,total_reviews,reviews
0,ChIJp1t0B3rBhkcRkhvz_gk4ARw,Mabuhay Restaurant,Filippina,"['restaurant', 'food', 'point_of_interest', 'e...","Bastioni di Porta Volta 9, 20121 Milano Italia",45.478721,9.185232,4.897793,1.511034,1722.0,"[{'bubbles': 'bubble_50', 'title': 'Piacevole ..."
1,ChIJNaMlrjfBhkcReLHYJXwZdZU,Parea Bistrot,NaN,"['restaurant', 'food', 'point_of_interest', 'e...","Via Vigna 6, 20123 Milano Italia",45.478721,9.185232,4.895327,NaN,214.0,"[{'bubbles': 'bubble_50', 'title': 'Locale sex..."
2,ChIJ-6W1E6bGhkcRR5BCiZUxEGg,Il Panino,"Italiana, Cibo di strada","['restaurant', 'food', 'point_of_interest', 'e...","Via Laghetto 7, 20122 Milano Italia",45.478721,9.185232,4.835329,1.000000,501.0,"[{'bubbles': 'bubble_50', 'title': 'Da brevett..."
3,ChIJAT5Z3wjBhkcRr-dUFcAhgX4,Cantiere Hambirreria,"Italiana, Americana","['restaurant', 'food', 'bar', 'point_of_intere...","Corso Garibaldi, 111, 20121 Milano Italia",45.478721,9.185232,4.675467,2.000000,1553.0,"[{'bubbles': 'bubble_50', 'title': 'SUPER!', '..."
4,ChIJ_4UjHuXBhkcRQXJ5r5hdRHs,BLUE M Milano - Bottega Marchigiana,Italiana,"['restaurant', 'food', 'point_of_interest', 'e...",Via Lanzone 27 Sant'Ambrogio - Università Catt...,45.478721,9.185232,4.889600,1.000000,250.0,"[{'bubbles': 'bubble_50', 'title': 'Le Marche ..."
...,...,...,...,...,...,...,...,...,...,...,...
3595,ChIJeyYJ7-bGhkcR32WFx77-Al4,D'sumare Restaurante & Polleria,"Peruviana, Cibo di strada","['restaurant', 'point_of_interest', 'food', 'e...","Via Padova 31, 20127 Milano Italia",45.478721,9.185232,NaN,NaN,1289.0,[]
3596,ChIJoZRSKbzHhkcR21lB-b6HT4M,Radhuni Indian Restaurant,"Indiana, Fast food","['restaurant', 'point_of_interest', 'food', 'e...","Via Predabissi angolo via Padova, Milano Italia",45.478721,9.185232,NaN,NaN,16.0,[]
3597,ChIJEymO4ePGhkcR2Mx0NtAsQME,Cleopatra Pizza,"Pizza, Fast food","['restaurant', 'point_of_interest', 'food', 'e...","Via Padova, 29, 20127 Milano Italia",45.478721,9.185232,NaN,NaN,34.0,[]
3598,ChIJN9AcT6LDhkcRPweDZ_3Hnqw,Davet Turkish Kebap Pizzeria Grill,"Pizza, Fast food","['restaurant', 'food', 'point_of_interest', 'e...","Piazzale Stazione Genova, 2, 20144 Milano Italia",45.478721,9.185232,NaN,1.000000,337.0,[]


PER METTERE SOLO I NODI IN NEO4J. 
POI LE RELAZIONI LE CREO CON DELLE QUESRY CIPHER DIRETTAMENTE SU NEO4J.

In [33]:
# !pip install neo4j
from neo4j import GraphDatabase

# Connect to Neo4j database
driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'progetto'))
session = driver.session()



# Iterate over the rows of the DataFrame
for index, row in df_integrated.iterrows():
    # Extract the data from the row
    name = row['name']
    rating = row['rating']
    total_reviews = row['total_reviews']
    cook_type = row['cook_type']
    price_level = row['price_level']
    address = row['address']
    latitude = row['latitude']
    longitude = row['longitude']

    # Create a Neo4j node for the restaurant
    session.run(
        """
        CREATE (restaurant:Restaurant {name: $name, rating: $rating, total_reviews: $total_reviews,
        cook_type: $cook_type, price_level: $price_level, address: $address, latitude: $latitude, longitude: $longitude})
        """,
        name=name,
        rating=rating,
        total_reviews=total_reviews,
        cook_type=cook_type,
        price_level=price_level,
        address=address,
        latitude=latitude,
        longitude=longitude
    )

session.close()
driver.close()


mongodb+srv://mmondini11:<password>@cluster0.xzuticb.mongodb.net/

***per cancellare LE RELAZIONI ma lasciare i nodi caricati*** :
MATCH (:Restaurant)-[r:RECOMMENDS]-(:Restaurant) DELETE r
 
***PER CANCELLARE A BATCH***
optional match (n)-[p:SHARES_CUISINE]->() with n,p LIMIT 300000  delete p


***matchare il singolo ristorante***
MATCH (r1:Restaurant {name: 'Say Cheese Bistrot'})-[:RECOMMENDS]->(rec:Restaurant)
RETURN r1, COLLECT(rec) AS recommendations


***cancellare tutto***
MATCH (n)
DETACH DELETE n


    RACCOMANDAZIONI USANDO TUTTI I PARAMETRI A DISPOSIZIONE.

    *NON MODIFICARE LA SINTASSI, MODIFICARE SOLO I PARAMETRI A PAICERE*


The [0..2] range SERVE PER CONSIDERARE I PRIMI TRE
SE VOGLIAMO UN NUMERO SPECIFICO AGGIUNGIAMO ----------LIMIT 3-------------
TO INTEGER VA PRIMA DI OGNI NUMERO


DA FARE: CERCARE DEI PARAMETRI MIGLIORI E NON A CASO

**QUI UNA CUCINE BASTA CHE MATCH CON ALMENO UNA CUCINA DELLA RACCOMANDAZIONE  E DISTANZA MAX 3KM, in più c'è la clausoa fast food**




MATCH (r1:Restaurant), (r2:Restaurant)
WHERE r1 <> r2 
  AND ((r1.cook_type = 'Fast Food' AND r2.cook_type = 'Fast Food') OR
       (r1.cook_type <> 'Fast Food' AND r2.cook_type <> 'Fast Food' AND
        (ANY(cuisine IN split(r1.cook_type, ',') WHERE r2.cook_type CONTAINS cuisine) OR
         ANY(cuisine IN split(r2.cook_type, ',') WHERE r1.cook_type CONTAINS cuisine))))
  AND toInteger(r1.price_level) >= toInteger(r2.price_level) - 1 AND toInteger(r1.price_level) <= toInteger(r2.price_level) + 1
  AND toFloat(r1.rating) >= toFloat(r2.rating) - 0.5 AND toFloat(r1.rating) <= toFloat(r2.rating) + 0.5
  AND toInteger(r1.total_reviews) >= toInteger(r2.total_reviews) - 50 AND toInteger(r1.total_reviews) <= toInteger(r2.total_reviews) + 50
  AND toFloat(r1.latitude) IS NOT NULL AND toFloat(r1.longitude) IS NOT NULL
  AND toFloat(r2.latitude) IS NOT NULL AND toFloat(r2.longitude) IS NOT NULL
  AND toFloat(r1.latitude) >= -90 AND toFloat(r1.latitude) <= 90
  AND toFloat(r1.longitude) >= -180 AND toFloat(r1.longitude) <= 180
  AND toFloat(r2.latitude) >= -90 AND toFloat(r2.latitude) <= 90
  AND toFloat(r2.longitude) >= -180 AND toFloat(r2.longitude) <= 180
  AND distance(point({ latitude: toFloat(r1.latitude), longitude: toFloat(r1.longitude) }), point({ latitude: toFloat(r2.latitude), longitude: toFloat(r2.longitude) })) <= 3000
WITH r1, r2, rand() AS random
ORDER BY random
WITH r1, COLLECT(r2) AS recommendedRestaurants
FOREACH (recRestaurant IN recommendedRestaurants[0..2] |
  MERGE (r1)-[:RECOMMENDS]->(recRestaurant)
)
RETURN r1, COLLECT(recommendedRestaurants[0..2]) AS recommendations


In [18]:
# from neo4j import GraphDatabase

# # Connect to Neo4j database

# #DATA SCHEMA
# from neo4j import GraphDatabase

# # Connect to Neo4j database
# driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'progetto'))
# session = driver.session()

# # Iterate over the rows of the DataFrame
# for index, row in df_integrated.iterrows():
#     # Extract the data from the row
#     address = row['address']
#     name = row['name']

#     # Check if address is empty or NaN
#     if pd.isna(address) or not address:
#         continue

#     # Convert the address value to string
#     address = str(address)

#     # Create or match the Prestaurant node
#     session.run(
#         """
#         MERGE (p:Prestaurant {Name: $name})
#         """
#         ,
#         name=name
#     )

#     # Create or match the address node
#     session.run(
#         """
#         MERGE (t:Address {Name: $address})
#         """
#         ,
#         address=address
#     )

#     # Create the relationship between Prestaurant and address nodes
#     session.run(
#         """
#         MATCH (p:Prestaurant {Name: $name}), (t:Address {Name: $address})
#         CREATE (p)-[:IS_LOCATED]->(t)
#         """
#         ,
#         name=name,
#         address=address
#     )

# session.close()
# driver.close()




In [19]:
# from py2neo import Graph

# # Set up the graph connection
# graph = Graph("bolt://localhost:7687", auth=("neo4j", "maurino"))

# # Define the pagination parameters
# page_size = 100  # Number of results per page
# page_number = 1  # Page number to retrieve (starting from 1)

# # Calculate the skip value based on the page size and number
# skip = (page_number - 1) * page_size

# # Construct the modified query with pagination
# query9 = '''
# MATCH (r1:Restaurant), (r2:Restaurant)
# WHERE r1 <> r2 
#   AND ((r1.cook_type = 'Fast Food' AND r2.cook_type = 'Fast Food') OR
#        (r1.cook_type <> 'Fast Food' AND r2.cook_type <> 'Fast Food' AND
#         (ANY(cuisine IN split(r1.cook_type, ',') WHERE r2.cook_type CONTAINS cuisine) OR
#          ANY(cuisine IN split(r2.cook_type, ',') WHERE r1.cook_type CONTAINS cuisine))))
#   AND toInteger(r1.price_level) >= toInteger(r2.price_level) - 1 AND toInteger(r1.price_level) <= toInteger(r2.price_level) + 1
#   AND toFloat(r1.rating) >= toFloat(r2.rating) - 0.5 AND toFloat(r1.rating) <= toFloat(r2.rating) + 0.5
#   AND toInteger(r1.total_reviews) >= toInteger(r2.total_reviews) - 50 AND toInteger(r1.total_reviews) <= toInteger(r2.total_reviews) + 50
#   AND toFloat(r1.latitude) IS NOT NULL AND toFloat(r1.longitude) IS NOT NULL
#   AND toFloat(r2.latitude) IS NOT NULL AND toFloat(r2.longitude) IS NOT NULL
#   AND toFloat(r1.latitude) >= -90 AND toFloat(r1.latitude) <= 90
#   AND toFloat(r1.longitude) >= -180 AND toFloat(r1.longitude) <= 180
#   AND toFloat(r2.latitude) >= -90 AND toFloat(r2.latitude) <= 90
#   AND toFloat(r2.longitude) >= -180 AND toFloat(r2.longitude) <= 180
#   AND point.distance(point({ latitude: toFloat(r1.latitude), longitude: toFloat(r1.longitude) }), point({ latitude: toFloat(r2.latitude), longitude: toFloat(r2.longitude) })) <= 3000
# WITH r1, r2, rand() AS random
# ORDER BY random
# WITH r1, COLLECT(r2) AS recommendedRestaurants
# SKIP $skip
# LIMIT $limit
# RETURN r1, COLLECT(recommendedRestaurants[0..2]) AS recommendations
# '''

# # Execute the modified query with pagination
# page_results = graph.run(query9, skip=skip, limit=page_size).data()

# # Print the results of the current page
# for record in page_results:
#     print(record)


In [43]:
query7 = '''
MATCH (r1:Restaurant), (r2:Restaurant)
WHERE r1 <> r2 
  AND ((r1.cook_type = 'Fast Food' AND r2.cook_type = 'Fast Food') OR
       (r1.cook_type <> 'Fast Food' AND r2.cook_type <> 'Fast Food' AND
        (ANY(cuisine IN split(toString(r1.cook_type), ',') WHERE r2.cook_type CONTAINS cuisine) OR
         ANY(cuisine IN split(toString(r2.cook_type), ',') WHERE r1.cook_type CONTAINS cuisine))))
  AND toInteger(r1.price_level) >= toInteger(r2.price_level) - 0.5 AND toInteger(r1.price_level) <= toInteger(r2.price_level) + 0.5
  AND toFloat(r1.rating) >= toFloat(r2.rating) - 0.5 AND toFloat(r1.rating) <= toFloat(r2.rating) + 0.5
  AND toInteger(r1.total_reviews) >= toInteger(r2.total_reviews) - 30 AND toInteger(r1.total_reviews) <= toInteger(r2.total_reviews) + 30
  AND toFloat(r1.latitude) IS NOT NULL AND toFloat(r1.longitude) IS NOT NULL
  AND toFloat(r2.latitude) IS NOT NULL AND toFloat(r2.longitude) IS NOT NULL
  AND toFloat(r1.latitude) >= -90 AND toFloat(r1.latitude) <= 90
  AND toFloat(r1.longitude) >= -180 AND toFloat(r1.longitude) <= 180
  AND toFloat(r2.latitude) >= -90 AND toFloat(r2.latitude) <= 90
  AND toFloat(r2.longitude) >= -180 AND toFloat(r2.longitude) <= 180
  AND point.distance(point({ latitude: toFloat(r1.latitude), longitude: toFloat(r1.longitude) }), point({ latitude: toFloat(r2.latitude), longitude: toFloat(r2.longitude) })) <= 3000
WITH r1, r2, rand() AS random,
     abs(toInteger(r1.price_level) - toInteger(r2.price_level)) AS price_diff,
     abs(toFloat(r1.rating) - toFloat(r2.rating)) AS rating_diff,
     abs(toInteger(r1.total_reviews) - toInteger(r2.total_reviews)) AS review_diff
ORDER BY random
WITH r1, COLLECT(r2) AS recommendedRestaurants, 
     MIN(price_diff) AS min_price_diff, 
     MIN(rating_diff) AS min_rating_diff, 
     MIN(review_diff) AS min_review_diff
WITH r1, recommendedRestaurants, 
     CASE WHEN min_price_diff = 0 THEN 1 ELSE (abs(toInteger(r1.price_level) - toInteger(r1.price_level)) / min_price_diff) END AS price_score, 
     CASE WHEN min_rating_diff = 0 THEN 1 ELSE (abs(toFloat(r1.rating) - toFloat(r1.rating)) / min_rating_diff) END AS rating_score, 
     CASE WHEN min_review_diff = 0 THEN 1 ELSE (abs(toInteger(r1.total_reviews) - toInteger(r1.total_reviews)) / min_review_diff) END AS review_score
WITH r1, recommendedRestaurants, 
     (price_score + rating_score + review_score) AS total_score
ORDER BY total_score DESC
WITH r1, COLLECT(recommendedRestaurants[0..2]) AS recommendations
FOREACH (recRestaurant IN recommendations |
  FOREACH (rec IN recRestaurant |
    MERGE (r1)-[:RECOMMENDS]->(rec)
  )
)
RETURN r1, recommendations
'''


from py2neo import Graph
from py2neo.data import Node, Relationship
from pprint import pprint
graph = Graph("bolt://localhost:7687", auth=("neo4j", "progetto"))
graph.run(query7)
cursor = graph.run(query7)

In [37]:
query7 = '''
MATCH (r1:Restaurant), (r2:Restaurant)
WHERE r1 <> r2 
  AND ((r1.cook_type = 'Fast Food' AND r2.cook_type = 'Fast Food') OR
       (r1.cook_type <> 'Fast Food' AND r2.cook_type <> 'Fast Food' AND
        (ANY(cuisine IN split(toString(r1.cook_type), ',') WHERE r2.cook_type CONTAINS cuisine) OR
         ANY(cuisine IN split(toString(r2.cook_type), ',') WHERE r1.cook_type CONTAINS cuisine))))
  AND toInteger(r1.price_level) >= toInteger(r2.price_level) - 0.5 AND toInteger(r1.price_level) <= toInteger(r2.price_level) + 0.5
  AND toFloat(r1.rating) >= toFloat(r2.rating) - 0.5 AND toFloat(r1.rating) <= toFloat(r2.rating) + 0.5
  AND toInteger(r1.total_reviews) >= toInteger(r2.total_reviews) - 30 AND toInteger(r1.total_reviews) <= toInteger(r2.total_reviews) + 30
  AND toFloat(r1.latitude) IS NOT NULL AND toFloat(r1.longitude) IS NOT NULL
  AND toFloat(r2.latitude) IS NOT NULL AND toFloat(r2.longitude) IS NOT NULL
  AND toFloat(r1.latitude) >= -90 AND toFloat(r1.latitude) <= 90
  AND toFloat(r1.longitude) >= -180 AND toFloat(r1.longitude) <= 180
  AND toFloat(r2.latitude) >= -90 AND toFloat(r2.latitude) <= 90
  AND toFloat(r2.longitude) >= -180 AND toFloat(r2.longitude) <= 180
  AND point.distance(point({ latitude: toFloat(r1.latitude), longitude: toFloat(r1.longitude) }), point({ latitude: toFloat(r2.latitude), longitude: toFloat(r2.longitude) })) <= 3000
WITH r1, r2, rand() AS random,
     abs(toInteger(r1.price_level) - toInteger(r2.price_level)) AS price_diff,
     abs(toFloat(r1.rating) - toFloat(r2.rating)) AS rating_diff,
     abs(toInteger(r1.total_reviews) - toInteger(r2.total_reviews)) AS review_diff
ORDER BY random
WITH r1, COLLECT(r2) AS recommendedRestaurants, 
     MIN(price_diff) AS min_price_diff, 
     MIN(rating_diff) AS min_rating_diff, 
     MIN(review_diff) AS min_review_diff
WITH r1, recommendedRestaurants, 
     CASE WHEN min_price_diff = 0 THEN 1 ELSE (abs(toInteger(r1.price_level) - toInteger(r1.price_level)) / min_price_diff) END AS price_score, 
     CASE WHEN min_rating_diff = 0 THEN 1 ELSE (abs(toFloat(r1.rating) - toFloat(r1.rating)) / min_rating_diff) END AS rating_score, 
     CASE WHEN min_review_diff = 0 THEN 1 ELSE (abs(toInteger(r1.total_reviews) - toInteger(r1.total_reviews)) / min_review_diff) END AS review_score
WITH r1, recommendedRestaurants, 
     (price_score + rating_score + review_score) AS total_score
ORDER BY total_score DESC
WITH r1, COLLECT(recommendedRestaurants[0..2]) AS recommendations
FOREACH (recRestaurant IN recommendations |
  FOREACH (rec IN recRestaurant |
    MERGE (r1)-[:RECOMMENDS]->(rec)
  )
)
RETURN r1, recommendations
'''


from py2neo import Graph
from py2neo.data import Node, Relationship
from pprint import pprint
graph = Graph("bolt://localhost:7687", auth=("neo4j", "progetto"))
graph.run(query7)
cursor = graph.run(query7)

In [21]:
# from neo4j import GraphDatabase

# # Connect to Neo4j database
# driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'progetto'))
# session = driver.session()

# # Iterate over the rows of the DataFrame
# for index, row in df_integrated.iterrows():
#     # Extract the data from the row
#     address = row['address']
#     name = row['name']

#     # Check if address is empty or NaN
#     if pd.isna(address) or not address:
#         continue

#     # Convert the address value to string
#     address = str(address)

#     # Create or match the Prestaurant node
#     session.run(
#         """
#         MERGE (p:Prestaurant {Name: $name})
#         """
#         ,
#         name=name
#     )

#     # Create or match the address node
#     session.run(
#         """
#         MERGE (t:Address {Name: $address})
#         """
#         ,
#         address=address
#     )

#     # Create the relationship between Prestaurant and address nodes
#     session.run(
#         """
#         MATCH (p:Prestaurant {Name: $name}), (t:Address {Name: $address})
#         CREATE (p)-[:IS_LOCATED]->(t)
#         """
#         ,
#         name=name,
#         address=address
#     )

# # Run the recommendation query
# recommendation_query = '''
# MATCH (r1:Restaurant), (r2:Restaurant)
# WHERE r1 <> r2 
#   AND ((r1.cook_type = 'Fast Food' AND r2.cook_type = 'Fast Food') OR
#        (r1.cook_type <> 'Fast Food' AND r2.cook_type <> 'Fast Food' AND
#         (ANY(cuisine IN split(toString(r1.cook_type), ',') WHERE r2.cook_type CONTAINS cuisine) OR
#          ANY(cuisine IN split(toString(r2.cook_type), ',') WHERE r1.cook_type CONTAINS cuisine))))
#   AND toInteger(r1.price_level) >= toInteger(r2.price_level) - 0.5 AND toInteger(r1.price_level) <= toInteger(r2.price_level) + 0.5
#   AND toFloat(r1.rating) >= toFloat(r2.rating) - 0.5 AND toFloat(r1.rating) <= toFloat(r2.rating) + 0.5
#   AND toInteger(r1.total_reviews) >= toInteger(r2.total_reviews) - 30 AND toInteger(r1.total_reviews) <= toInteger(r2.total_reviews) + 30
#   AND toFloat(r1.latitude) IS NOT NULL AND toFloat(r1.longitude) IS NOT NULL
#   AND toFloat(r2.latitude) IS NOT NULL AND toFloat(r2.longitude) IS NOT NULL
#   AND toFloat(r1.latitude) >= -90 AND toFloat(r1.latitude) <= 90
#   AND toFloat(r1.longitude) >= -180 AND toFloat(r1.longitude) <= 180
#   AND toFloat(r2.latitude) >= -90 AND toFloat(r2.latitude) <= 90
#   AND toFloat(r2.longitude) >= -180 AND toFloat(r2.longitude) <= 180
#   AND point.distance(point({ latitude: toFloat(r1.latitude), longitude: toFloat(r1.longitude) }), point({ latitude: toFloat(r2.latitude), longitude: toFloat(r2.longitude) })) <= 3000
# WITH r1, r2, rand() AS random,
#      abs(toInteger(r1.price_level) - toInteger(r2.price_level)) AS price_diff,
#      abs(toFloat(r1.rating) - toFloat(r2.rating)) AS rating_diff,
#      abs(toInteger(r1.total_reviews) - toInteger(r2.total_reviews)) AS review_diff
# ORDER BY random
# WITH r1, COLLECT(r2) AS recommendedRestaurants, 
#      MIN(price_diff) AS min_price_diff, 
#      MIN(rating_diff) AS min_rating_diff, 
#      MIN(review_diff) AS min_review_diff
# WITH r1, recommendedRestaurants, 
#      CASE WHEN min_price_diff = 0 THEN 1 ELSE (abs(toInteger(r1.price_level) - toInteger(r1.price_level)) / min_price_diff) END AS price_score, 
#      CASE WHEN min_rating_diff = 0 THEN 1 ELSE (abs(toFloat(r1.rating) - toFloat(r1.rating)) / min_rating_diff) END AS rating_score, 
#      CASE WHEN min_review_diff = 0 THEN 1 ELSE (abs(toInteger(r1.total_reviews) - toInteger(r1.total_reviews)) / min_review_diff) END AS review_score
# WITH r1, recommendedRestaurants, 
#      (price_score + rating_score + review_score) AS total_score
# ORDER BY total_score DESC
# WITH r1, COLLECT(recommendedRestaurants[0..2]) AS recommendations
# FOREACH (recRestaurant IN recommendations |
#   FOREACH (rec IN recRestaurant |
#     MERGE (r1)-[:RECOMMENDS]->(rec)
#   )
# )
# RETURN r1, recommendations
# '''

# results = session.run(recommendation_query)

# # Process the results
# for record in results:
#     r1 = record['r1']
#     recommendations = record['recommendations']
    
#     # Process and display the recommendation results as needed
    
# session.close()
# driver.close()


In [40]:
query8 = '''

MATCH (r1:Restaurant), (r2:Restaurant)
WHERE r1 <> r2 
  AND (
    (r1.cook_type = 'Italiana' AND r2.cook_type = 'Italiana') OR
    (r1.cook_type = 'Italiana' AND r2.cook_type = 'Italiana' AND size(split(r1.cook_type, ',')) > 1)
  )
  AND toInteger(r1.price_level) >= toInteger(r2.price_level) - 0.5 AND toInteger(r1.price_level) <= toInteger(r2.price_level) + 0.5
  AND toFloat(r1.rating) >= toFloat(r2.rating) - 0.5 AND toFloat(r1.rating) <= toFloat(r2.rating) + 0.5
  AND toInteger(r1.total_reviews) >= toInteger(r2.total_reviews) - 30 AND toInteger(r1.total_reviews) <= toInteger(r2.total_reviews) + 30
  AND toFloat(r1.latitude) IS NOT NULL AND toFloat(r1.longitude) IS NOT NULL
  AND toFloat(r2.latitude) IS NOT NULL AND toFloat(r2.longitude) IS NOT NULL
  AND toFloat(r1.latitude) >= -90 AND toFloat(r1.latitude) <= 90
  AND toFloat(r1.longitude) >= -180 AND toFloat(r1.longitude) <= 180
  AND toFloat(r2.latitude) >= -90 AND toFloat(r2.latitude) <= 90
  AND toFloat(r2.longitude) >= -180 AND toFloat(r2.longitude) <= 180
  AND point.distance(point({ latitude: toFloat(r1.latitude), longitude: toFloat(r1.longitude) }), point({ latitude: toFloat(r2.latitude), longitude: toFloat(r2.longitude) })) <= 3000
WITH r1, r2, rand() AS random,
     abs(toInteger(r1.price_level) - toInteger(r2.price_level)) AS price_diff,
     abs(toFloat(r1.rating) - toFloat(r2.rating)) AS rating_diff,
     abs(toInteger(r1.total_reviews) - toInteger(r2.total_reviews)) AS review_diff
ORDER BY random
WITH r1, COLLECT(r2) AS recommendedRestaurants, 
     MIN(price_diff) AS min_price_diff, 
     MIN(rating_diff) AS min_rating_diff, 
     MIN(review_diff) AS min_review_diff
WITH r1, recommendedRestaurants, 
     CASE WHEN min_price_diff = 0 THEN 1 ELSE (abs(toInteger(r1.price_level) - toInteger(r1.price_level)) / min_price_diff) END AS price_score, 
     CASE WHEN min_rating_diff = 0 THEN 1 ELSE (abs(toFloat(r1.rating) - toFloat(r1.rating)) / min_rating_diff) END AS rating_score, 
     CASE WHEN min_review_diff = 0 THEN 1 ELSE (abs(toInteger(r1.total_reviews) - toInteger(r1.total_reviews)) / min_review_diff) END AS review_score
WITH r1, recommendedRestaurants, 
     (price_score + rating_score + review_score) AS total_score
ORDER BY total_score DESC
WITH r1, COLLECT(recommendedRestaurants[0..2]) AS recommendations
FOREACH (recRestaurant IN recommendations |
  FOREACH (rec IN recRestaurant |
    MERGE (r1)-[:RECOMMENDS]->(rec)
  )
)
RETURN r1, recommendations


'''




from py2neo import Graph
from py2neo.data import Node, Relationship
from pprint import pprint
graph = Graph("bolt://localhost:7687", auth=("neo4j", "progetto"))
graph.run(query8)
cursor = graph.run(query8)